In [110]:
import pandas as pd
import pandas
import re 
r=open('names.txt')
cols1=r.read()
cols=cols1.split('\n')
columns=[]
for i in cols:
    value=i.split(':')[0]
    columns.append(value)
columns=columns[-15:-1]
columns.append('high_income')
for i in columns:
    index=columns.index(i)
    i=i.replace('-','_')
    columns[index]=i
income=pd.read_csv('adult.txt')
income.columns=columns








In [119]:
# Convert a single column from text categories to numbers
col = pandas.Categorical.from_array(income["workclass"])
income["workclass"] = col.codes
col = pandas.Categorical.from_array(income["education"])
income["education"] = col.codes
col = pandas.Categorical.from_array(income["marital_status"])
income["marital_status"] = col.codes
col = pandas.Categorical.from_array(income["occupation"])
income["occupation"] = col.codes
col = pandas.Categorical.from_array(income["relationship"])
income["relationship"] = col.codes
col = pandas.Categorical.from_array(income["race"])
income["race"] = col.codes
col = pandas.Categorical.from_array(income["sex"])
income["sex"] = col.codes
col = pandas.Categorical.from_array(income["native_country"])
income["native_country"] = col.codes
col = pandas.Categorical.from_array(income["high_income"])
income["high_income"] = col.codes
income.head()

/Users/Jaspinder/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  from ipykernel import kernelapp as app
/Users/Jaspinder/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
/Users/Jaspinder/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
/Users/Jaspinder/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
/Users/Jaspinder/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
/Users/Jaspinder/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
/Users/Jaspinder/anac

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
3,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0
4,37,4,284582,12,14,2,4,5,4,0,0,0,40,39,0


**8: Overview Of Data Set Entropy**

Now that we have a high-level view of how decision trees work, let's explore the details and learn how to perform the splits.

We'll use a specific measure to figure out which variables we should split nodes on. Post-split, we'll have two data sets, each containing the rows from one branch of the split.

Because we're trying to reach the leaves having only 1s or only 0s in high_income, each split will need to get us closer to that goal.

When we split, we'll try to separate as many 0s from 1s in the high_income column as we can. In order to do this, we need a metric for how "together" the different values in the high_income column are.

Data scientists commonly use a metric called entropy for this purpose. Entropy refers to disorder. The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the high_income column would have low entropy.

Entropy, which is not to be confused with entropy from physics, comes from information theory. Information theory is based on probability and statistics, and deals with the transmission, processing, utilization, and extraction of information. A key concept in information theory is the notion of a bit of information. One bit of information is one unit of information.

We can represent a bit of information as a binary number because it either has the value 1 or 0. Suppose there's an equal probability of tomorrow being sunny (1) or not sunny (0). If I tell you that it will be sunny, I've given you one bit of information.

We can also think of entropy in terms of information. If we flip a coin where both sides are heads, we know upfront that the result will be heads. We gain no new information by flipping the coin, so entropy is 0. On the other hand, if the coin has a heads side and a tails side, there's a 50% probability that it will land on either. Thus, flipping the coin gives us one bit of information -- which side the coin landed on.

Entropy can be much more complex, especially when we get to cases with more than two possible outcomes, or differential probabilities. A deep understanding of entropy isn't necessary for constructing decision trees, however. If you'd like, you can read more about entropy at Wikipedia.

The formula for entropy looks like this:

−∑ci=1P(xi)logbP(xi)

We iterate through each unique value in a single column (in this case, high_income), and assign it to ii. We then compute the probability of that value occurring in the data (P(xi)P(xi)). Next we do some multiplication, and sum all of the values together. bb is the base of the logarithm. We commonly use the value 2 for this, but we can also set it to 10 or another value.

Let's say we have this data:


age    high_income
25     1
50     1
30     0
50     0
80     1
We could compute its entropy like this:

−∑i=1cP(xi)logbP(xi)

−((2/5∗log22/5)+(3/5∗log23/5))

−(−0.5287712379549449+−0.44217935649972373)

.97

We get less than one "bit" of information -- only .97 -- because there are slightly more 1s in the sample data than 0s. This means that if we were predicting a new value, we could guess that the answer is 1 and be right more often than wrong (because there's a .6 probability of the answer being 1). Due to this prior knowledge, we gain less than a full "bit" of information when we observe a new value.

In [175]:
def cal_entropy(df,target_col_name):
    zero_prob=(df[target_col_name].value_counts()[0])/len(df)
    one_prob=(df[target_col_name].value_counts()[1])/len(df)
    target_entropy=-1*((zero_prob)*(math.log(zero_prob,2))+(one_prob)*(math.log(one_prob,2)))
    return target_entropy
income_entropy=cal_entropy(income,'high_income')
income_entropy
    

0.79639620675821887

**9: Information Gain**
    
    
We'll need a way to go from computing entropy to figuring out which variable to split on. We can do this using information gain, which tells us which split will reduce entropy the most.

Here's the formula for information gain:

IG(T,A)=Entropy(T)−∑v∈A|Tv||T|⋅Entropy(Tv)

It may look complicated, but we'll break it down. We're computing information gain (IG) for a given target variable (T), as well as a given variable we want to split on (A).

To compute it, we first calculate the entropy for TT. Then, for each unique value vv in the variable AA, we compute the number of rows in which AA takes on the value vv, and divide it by the total number of rows. Next, we multiply the results by the entropy of the rows where AA is vv. We add all of these subset entropies together, then subtract from the overall entropy to get information gain.

Here's an alternate explanation. We're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If the result is positive, we've lowered entropy with our split. The higher the result is, the more we've lowered entropy.

One strategy for constructing trees is to create as many branches at each node as there are unique values for the variable we're splitting on. So if the variable has three or four values, we'd end up with three or four branches. This approach usually involves more complexity than it's worth and doesn't improve prediction accuracy, but it's worth knowing about.

To simplify the calculation of information gain and make splits simpler, we won't do it for each unique value. We'll find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.

Here's an example that uses the same data set we worked with earlier:


age    high_income
25     1
50     1
30     0
50     0
80     1
Let's say we wanted to split this data set based on age. First, we calculate the median age, which is 50. Then, we assign any row with a value less than or equal to the median age the value 0 (in a new column named split_age), and the other rows 1.


age    high_income    split_age
25     1              0
50     1              0
30     0              0
50     0              0
80     1              1
Now we compute entropy:

IG(T,A)=Entropy(T)−∑v∈A(|Tv|/|T|)⋅Entropy(Tv)

.97−(((4/5)∗−(1/2∗log21/2+1/2∗log21/2))+−(1/5∗(0∗log20+1∗log21)))

.97−((4/5)∗−(−.5+−.5))+(1/5∗−(0+1∗0))

.97−(4/5).169

We end up with .17, which means that we gain .17 bits of information by splitting our data set on the age variable.

In [200]:
###Compute the information gain for splitting on the age column of income.
##First, compute the median of age.
##Then, assign anything less than or equal to the median to the left branch, and anything greater than the median to the right branch.
###Compute the information gain and assign it to age_information_gain.
def cal_entropy(df,target_col_name):
    zero_prob=(df[target_col_name].value_counts()[0])/len(df)
    one_prob=(df[target_col_name].value_counts()[1])/len(df)
    target_entropy=-1*((zero_prob)*(math.log(zero_prob,2))+(one_prob)*(math.log(one_prob,2)))
    return target_entropy

def information_gain(df,col_name,target_col_name):
    median_col=df[col_name].median()
    split_col_name='split_'+col_name
    df[split_col_name]=df[col_name]>median_col
    zero_split_col_prob=df[split_col_name].value_counts()[0]/len(df)
    one_split_col_prob=df[split_col_name].value_counts()[1]/len(df)
    df_split_col_0=df[df[split_col_name]==0]
    df_split_col_0_target_entropy=cal_entropy(df_split_col_0,target_col_name)
    df_split_col_1=df[df[split_col_name]==1]
    df_split_col_1_target_entropy=cal_entropy(df_split_col_0,target_col_name)
    IG_summation_value=zero_split_col_prob*df_split_col_0_target_entropy+one_split_col_prob*df_split_col_1_target_entropy
    col_information_gain=cal_entropy(df,target_col_name)-IG_summation_value
    return col_information_gain

age_information_gain=information_gain(income,'age','high_income')
age_information_gain

    

0.22410907689234416

In [216]:
###Create a list called information_gains.
###It should contain, in order, the information gain from splitting on these columns: age, workclass, education_num, marital_status, occupation, relationship, race, sex, hours_per_week, native_country.
##Find the highest value in the information_gains list, and assign the corresponding column name to highest_gain.

columns = ["age",'native_country','workclass','marital_status','occupation','hours_per_week','relationship']
information_gains=[]
for i in columns:
    value=information_gain(income,i,'high_income')
    information_gains.append(value)
print(information_gains)
max(information_gains)
    



[0.22410907689234416, -0.0004581269315567793, 0.031508157028117822, -0.15314207819252945, 0.030877405216549692, 0.1309291786180935, -0.10259225780768544]


0.22410907689234416